In [2]:
import dynet as dy

In [3]:
import numpy as np

In [4]:
pc = dy.ParameterCollection()

In [5]:
dy.load("save/network1143",pc)

[LookupParameter /_1,
 Parameter /_0,
 Parameter /W00,
 Parameter /b00,
 Parameter /W01,
 Parameter /b01,
 Parameter /W00_1,
 Parameter /b00_1,
 Parameter /W01_1,
 Parameter /b01_1,
 Parameter /W00_2,
 Parameter /b00_2,
 Parameter /W01_2,
 Parameter /b01_2]

In [6]:
pc.parameters_list()[0].npvalue().shape


(200, 300)

In [7]:
cd code

/mnt/c/Shahar/BarIlan/NLP-courses/89687-DL/Assignment4/code


In [8]:
import decomposable as de
import parsers 
GLOVE_FILE = '../glove_filtered00.txt'
SNLI_TRAIN = '../snli_1.0/snli_1.0_train_stripped.txt'
SNLI_DEV = '../snli_1.0/snli_1.0_dev_stripped.txt'
max_cases = 0
reload(de)
reload(de.net)

<module 'network' from 'network.pyc'>

In [9]:
train_data, labels = parsers.load_snli(SNLI_TRAIN, max_cases)
dev_data, labels = parsers.load_snli(SNLI_DEV, labels=labels)
    

In [10]:
with open(GLOVE_FILE, 'rt') as a:
    glove = parsers.glove_embeddings(a)


['La', 'Lo', 'glassblowers', 'want', 'absolute', 'wracked', 'LA', 'travel', 'shirked', 'copious']


In [11]:
matrices = pc.lookup_parameters_list() + pc.parameters_list()

In [12]:
myde = de.decomposable(glove, 300,200, 3, pc, matrices)

loading pretrained inputs
dimension reducer [1]
attend [2, 3, 4, 5]
compare [6, 7, 8, 9]
aggregate [10, 11, 12, 13]
layer sizes [200, 200, 200]
layer sizes [400, 200, 200]
layer sizes [400, 400, 3]


In [13]:
print "attend", [x.npvalue().shape for x in myde.attend.params["W"]]
print "attend", [x.npvalue().shape for x in myde.attend.params["b"]]
print "compare", [x.npvalue().shape for x in myde.compare.params["W"]]
print "aggregate", [x.npvalue().shape for x in myde.aggregate.params["W"]]

attend [(200, 200), (200, 200)]
attend [(200,), (200,)]
compare [(200, 400), (200, 200)]
aggregate [(400, 400), (3, 400)]


In [14]:
[(i, x.npvalue().shape)  for i, x in enumerate(matrices[1:],1)] 

[(1, (200, 300)),
 (2, (200, 200)),
 (3, (200,)),
 (4, (200, 200)),
 (5, (200,)),
 (6, (200, 400)),
 (7, (200,)),
 (8, (200, 200)),
 (9, (200,)),
 (10, (400, 400)),
 (11, (400,)),
 (12, (3, 400)),
 (13, (3,))]

In [15]:
print "attend", myde.attend.layer_sizes
print "compare", myde.compare.layer_sizes
print "aggregate", myde.aggregate.layer_sizes

attend [200, 200, 200]
compare [400, 200, 200]
aggregate [400, 400, 3]


In [16]:
def evaluate_accuracy(deco, dataset):
    i = 0
    mloss = 0.
    goods = 0
    for x, y in dataset:
        i = i + 1
        mloss = mloss + deco.eval_loss(x, y).npvalue()
        good = y == deco.last_case_class
        goods += int(good)
        dy.renew_cg()
        if i % 10000 == 1000:
            print "{} cases".format(i)
    return goods, i, mloss


In [17]:
ret = evaluate_accuracy(myde, dev_data)
print ret

1000 cases
(4193, 9842, array([13118.62008694]))


In [98]:
myde.params["E"].npvalue().shape

ValueError: Failed dimension check in MatrixMultiply::forward

In [18]:
4193./9842


0.4260312944523471